In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.backends.backend_pdf as bkd
import matplotlib.ticker as ticker

In [ ]:
df = pd.read_csv("../output/2024-01-18.csv", parse_dates=["day", "asleep_datetime_utc"])
df

In [ ]:
df['asleep_datetime'] = df['asleep_datetime_utc'].dt.tz_convert('Europe/Paris')
df

In [ ]:
df['hour'] = df['asleep_datetime'].dt.hour
df

In [ ]:
ax = df[df.day > '2023-12-18'].plot(x='day', y='hour', kind='bar')
plt.yticks(np.arange(0, df['hour'].max() + 1, 1))
plt.show()

In [ ]:
df = df.assign(norm_hour = lambda x: x['hour'].apply(lambda y: y if y > 12 else y + 24))
df

In [ ]:
mean_norm_hour = df['norm_hour'].mean()
ax = df[df.day > '2023-12-18'].plot(x='day', y='norm_hour', kind='bar', alpha=0.7, label='norm_hour')
ax.axhline(mean_norm_hour, color='red', linestyle='dashed', label='mean_norm_hour')
plt.yticks(np.arange(0, df['norm_hour'].max() + 1, 1))
plt.legend()
plt.show()

In [ ]:
# Calculate the global mean norm_hour
global_mean_norm_hour = df['norm_hour'].mean()

# Get unique years and months
unique_years = df['day'].dt.year.unique()
unique_months = df['day'].dt.month.unique()

pdf_pages = bkd.PdfPages('2024-01-18.pdf')

df['day_label'] = df['day'].dt.strftime('%Y-%m-%d')

# Loop over unique years
for year in unique_years:
    # Loop over unique months
    for month in unique_months:
        # Filter DataFrame for current year and month
        df_year_month = df[(df['day'].dt.year == year) & (df['day'].dt.month == month)]

        if df_year_month.empty:
            continue

        # Calculate the mean norm_hour for the current month
        mean_norm_hour_monthly = df_year_month['norm_hour'].mean()

        # Create a new figure for each year and month
        fig, ax = plt.subplots()

        # Plot 'norm_hour' as a bar chart
        df_year_month.plot(x='day_label', y='norm_hour', kind='bar', alpha=0.7, label='norm_hour', ax=ax)

        # Add a horizontal line at the global mean norm_hour
        ax.axhline(global_mean_norm_hour, color='green', linestyle='dashdot', label='Global Mean')

        # Add a horizontal line at the mean norm_hour for the current month
        ax.axhline(mean_norm_hour_monthly, color='red', linestyle='dashed', label='Monthly Mean')

        plt.yticks(np.arange(0, df['norm_hour'].max() + 1, 1))
        plt.legend()
        plt.title(f'{pd.to_datetime(df_year_month["day"].iloc[0])} - {pd.to_datetime(df_year_month["day"].iloc[-1])}')

        # Apply tight layout
        plt.tight_layout()

        pdf_pages.savefig(fig)
        plt.clf()

# Close the PDF file
pdf_pages.close()

In [ ]:
df_year_month.info()